In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

oncall_mcp_server_image = "oncall-mcp-server"
oncall_mcp_server_src = "src/oncall/mcp-server"

github_mcp_server_image = "github-mcp-server"
github_mcp_server_src = "src/github/mcp-server"

servicenow_mcp_server_image = "servicenow-mcp-server"
servicenow_mcp_server_src = "src/servicenow/mcp-server"
servicenow_instance_name = "" # Add here the name of your ServiceNow instance, e.g. "businessname-dev". Leave empty if you don't want to use ServiceNow.

utils.print_ok('Notebook initialized')

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    oncall_containerapp_resource_name = utils.get_deployment_output(output, 'oncallMCPServerContainerAppResourceName', 'Oncall Container App Resource Name')

    a2a_weather_containerapp_resource_name = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppResourceName', 'A2A (Weather) Agent Container App Resource Name')
    a2a_oncall_containerapp_resource_name = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppResourceName', 'A2A (Oncall) Agent Container App Resource Name')

    a2a_weather_a2a_agent_ep = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppFQDN', 'A2A (Weather) Agent Endpoint')
    a2a_oncall_a2a_agent_ep = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppFQDN', 'A2A (Oncall) Agent Endpoint')

inference_api_path = ""
inference_api_version = "2025-03-01-preview"

In [ ]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin

async def _safe_disconnect(plugin: MCPSsePlugin) -> None:
    """Handle SK method differences across versions."""
    for method_name in ("disconnect", "close", "aclose"):
        method = getattr(plugin, method_name, None)
        if method:
            maybe_coro = method()
            if asyncio.iscoroutine(maybe_coro):
                await maybe_coro
            return

async def build_agent():
    # Connect the agent to Azure OpenAI
    service = AzureChatCompletion(
            endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
            api_key=apim_subscription_key,
            api_version=inference_api_version,                
            deployment_name=openai_model_name  # Use the first model from the models_config
        )

    # Attach a remote MCP plugin the agent can call during reasoning
    # (e.g., a weather or tools server you already host elsewhere)
    weather_plugin = MCPSsePlugin(
        name="Weather",
        url=f"{apim_resource_gateway_url}/weather/sse",
        description="Remote Weather MCP Plugin via SSE",
    )

    await weather_plugin.connect()

    agent = ChatCompletionAgent(
        service=service,
        name="WeatherAgent",
        instructions=(
            "You are a helpful assistant. "
            "Use the 'Weather' plugin when the user asks about weather or locations. "
            "Cite the source if appropriate."
        ),
        plugins=[weather_plugin],
    )

    return agent


In [ ]:
import argparse
import logging
from typing import Any, Literal

from starlette.responses import Response
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig

async def run(transport: Literal["sse", "stdio", "http"] = "stdio", port: int | None = None) -> None:
    kernel = await build_agent()

    @kernel_function()
    def echo_function(message: str, extra: str = "") -> str:
        """Echo a message as a function"""
        return f"Function echo: {message} {extra}"

    server = kernel.as_mcp_server(server_name="sk")

    if transport == "http" and port is not None:
        import contextlib
        from mcp.server.streamable_http_manager import StreamableHTTPSessionManager
        from mcp.server.fastmcp import FastMCP
        from starlette.types import Receive, Scope, Send
        from starlette.applications import Starlette
        from typing import AsyncIterator
        from starlette.routing import Mount
        import uvicorn

        session_manager = StreamableHTTPSessionManager(
            app=server,
            event_store=None,
            json_response=True,
            stateless=True,
            )
        async def handle_streamable_http(scope: Scope, receive: Receive, send: Send) -> None:
            await session_manager.handle_request(scope, receive, send)

        @contextlib.asynccontextmanager
        async def lifespan(app: Starlette) -> AsyncIterator[None]:
            """Context manager for session manager."""
            async with session_manager.run():
                try:
                    yield
                finally:
                    print("Application shutting down...")
        
        starlette_app = Starlette(
            debug=True,
                routes=[
                    Mount("/mcp", app=handle_streamable_http),
                ],
                lifespan=lifespan,
            )
        uvicorn.run(starlette_app, host="0.0.0.0", port=port)

    if transport == "sse" and port is not None:
        import uvicorn
        from mcp.server.sse import SseServerTransport
        from starlette.applications import Starlette
        from starlette.routing import Mount, Route

        sse = SseServerTransport("/messages/")

        async def handle_sse(request):
            async with sse.connect_sse(request.scope, request.receive, request._send) as (read_stream, write_stream):
                await server.run(read_stream, write_stream, server.create_initialization_options())
            return Response(status_code=204)  # <— important!

        starlette_app = Starlette(
            debug=True,
            routes=[
                Route("/sse", endpoint=handle_sse),
                Mount("/messages/", app=sse.handle_post_message),
            ],
        )
        import nest_asyncio
        nest_asyncio.apply() 

        uvicorn.run(starlette_app, host="0.0.0.0", port=port)  # nosec
        
    elif transport == "stdio":
        import anyio
        from mcp.server.stdio import stdio_server

        async def handle_stdin(stdin: Any | None = None, stdout: Any | None = None) -> None:
            async with stdio_server() as (read_stream, write_stream):
                await server.run(read_stream, write_stream, server.create_initialization_options())

        anyio.run(handle_stdin)

#### Blocking cell

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

## this a blocking call - it will block the running further cells
asyncio.run(run(transport="sse", port=9090), debug=True)  # Change transport to "stdio" if you want to run it in stdio mode

#### Non-blocking cell

In [ ]:
import asyncio

# Schedule your long-running coroutine without blocking the cell
loop = asyncio.get_event_loop()             # Jupyter’s loop
mcp_agent_task = loop.create_task(run(transport="sse", port=9090))
mcp_agent_task.set_name("mcp-agent")     # optional, helps with debugging

# (optional) surface exceptions instead of failing silently
def _report_done(t: asyncio.Task):
    try:
        t.result()
    except asyncio.CancelledError:
        print("mcp-agent: cancelled")
    except Exception as e:
        print("mcp-agent: crashed with:", repr(e))

mcp_agent_task.add_done_callback(_report_done)

print("Started in background:", mcp_agent_task)
# Now you can continue running other cells without waiting for the agent to finish

### SSE Transport MCP Server Agents

In [ ]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import SseMcpToolAdapter, SseServerParams, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

async def run_agent(url, prompt) -> None:
    # Create server params for the remote MCP service
    server_params = SseServerParams(
        url=url,
        headers={"Content-Type": "application/json"},
        timeout=30,  # Connection timeout in seconds
    )

    # Get all available tools
    tools = await mcp_server_tools(server_params)

    # Create an agent that can use the translation tool
    model_client = AzureOpenAIChatCompletionClient(azure_deployment=openai_model_name, model=openai_model_name,
                azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                api_key=apim_subscription_key,
                api_version=inference_api_version
    )
    agent = AssistantAgent(
        name="weather",
        model_client=model_client,
        reflect_on_tool_use=True,
        tools=tools, # type: ignore
        system_message="You are a helpful assistant.",
    )
    await Console(
        agent.run_stream(task=prompt)
    )

import nest_asyncio
nest_asyncio.apply()
asyncio.run(run_agent(f"http://127.0.0.1:9090/sse", "What's the weather in Lisbon, Cairo and London?"))


### Stop the server

In [ ]:
# Ask it to stop
weather_task.cancel()

# Let it handle cancellation and swallow the CancelledError
await asyncio.gather(weather_task, return_exceptions=True)
